## Get Playlists Data

Let us understand how to get YouTube Playlists Data using Python as programming language using REST APIs.

Here are the steps involved in getting the data.
* Make sure to get the token and build **youtube** object.
* We can get list of playlists using `youtube.playlists().list` with appropriate arguments.
* As per the table structure below we want playlist id, title, description, video count and privacy status.
  * Playlist id, title and description comes from snippet part.
  * Count comes from contentDetails part.
  * Privacy Status comes from status part.
* We need to pass all the 3 parts to part argument for `youtube.playlists().list` along with channel id.

```python
playlists = youtube.playlists().list(
    part='snippet,contentDetails,status',
    channelId='UCakdSIPsJqiOLqylgoYmwQg'
).execute()
```

* YouTube APIs return at max 50 results. We need to capture nextToken and invoke `list` function to get next 50 results.

```python
nextPageToken = None
playlist_details = []
while True:
    playlists = youtube.playlists().list(
        part = 'snippet,contentDetails,status',
        channelId='UCakdSIPsJqiOLqylgoYmwQg',
        pageToken = nextPageToken,
        maxResults = 50
    ).execute()

    for item in playlists['items']:
        playlist_details.append(
            (
                item['id'], item['snippet']['title'], 
                item['snippet']['description'],
                item['contentDetails']['itemCount'], 
                item['status']['privacyStatus']
            )
        )

    nextPageToken = playlists.get('nextPageToken')
    if not nextPageToken:
        break
```

### Playlists Table Structure

Here are the fields we have as part of playlists table.

|Column Name|Data Type|
|---|---|
|playlist_id|VARCHAR(50)|
|playlist_title|VARCHAR(200)|
|playlist_description|VARCHAR(5000)|
|playlist_video_count|INT|
|playlist_privacy_status|VARCHAR(30)|

In [1]:
%run 03_configure_youtube_apis.ipynb    

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=684254837377-fgfrvlp2d6sml1975u58c9q6osj79tkt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube&state=giF9jO3nmSy8mnWR2DkUrfajJYJzT6&prompt=consent&access_type=offline


Enter the authorization code:  4/1AY0e-g5SGAxZ7h8lRYN6_XQVhU-TdY2bTimoRQCP6m_XlgvZ1ZiyHV2UZUk


In [2]:
youtube

In [3]:
playlists = youtube.playlists().list(
    part='snippet,contentDetails,status',
    channelId=channel_id
).execute()

In [4]:
playlists['items'][0]

{'kind': 'youtube#playlist',
 'etag': 'ynWeVSyUjJ5BHb9dRC9cRtQoFXw',
 'id': 'PLf0swTFhTI8rkH4yIfoyTAheEGjWIRtPG',
 'snippet': {'publishedAt': '2020-12-21T13:06:58Z',
  'channelId': 'UCakdSIPsJqiOLqylgoYmwQg',
  'title': 'Mastering Python - A Comprehensive Python Course for beginners and experienced developers',
  'description': 'This course is primarily designed to learn Python as Programming language to build web or mobile applications, data engineering applications, automation etc.\n\nYou can access the code and material along with the videos over https://python.itversity.com\n\nAbout Python\n\nPython is one of the leading programming language. It is an open source database and used for different types of applications.\n\n* Web Applications\n* Mobile Applications\n* Data Engineering Applications\n* Server Automation\n* Troubleshooting and Debugging\n* Quality Assurance\n* Data Science based Applications\nand many many more\n\nIt is one of the top 3 programming languages for almost a 

In [5]:
nextPageToken = None
playlist_details = []
while True:
    playlists = youtube.playlists().list(
        part = 'snippet,contentDetails,status',
        channelId='UCakdSIPsJqiOLqylgoYmwQg',
        pageToken = nextPageToken,
        maxResults = 50
    ).execute()

    for item in playlists['items']:
        playlist_details.append(
            (
                item['id'], item['snippet']['title'], item['snippet']['description'],
                item['contentDetails']['itemCount'], item['status']['privacyStatus']
            )
        )

    nextPageToken = playlists.get('nextPageToken')
    if not nextPageToken:
        break

In [6]:
len(playlist_details)

156

In [7]:
playlist_details[0]

('PLf0swTFhTI8rkH4yIfoyTAheEGjWIRtPG',
 'Mastering Python - A Comprehensive Python Course for beginners and experienced developers',
 'This course is primarily designed to learn Python as Programming language to build web or mobile applications, data engineering applications, automation etc.\n\nYou can access the code and material along with the videos over https://python.itversity.com\n\nAbout Python\n\nPython is one of the leading programming language. It is an open source database and used for different types of applications.\n\n* Web Applications\n* Mobile Applications\n* Data Engineering Applications\n* Server Automation\n* Troubleshooting and Debugging\n* Quality Assurance\n* Data Science based Applications\nand many many more\n\nIt is one of the top 3 programming languages for almost a decade now along with Java and Java Script.\n\nCourse Details\n\nThis course is primarily designed to go through core programming using Python. It will serve as foundation for role specific course

In [8]:
def get_playlists(youtube, channel):
    nextPageToken = None
    playlists_details = []
    
    while True:
        playlists = youtube.playlists().list(
            part = 'snippet,contentDetails,status',
            channelId=channel,
            pageToken = nextPageToken,
            maxResults = 50
        ).execute()

        for item in playlists['items']:
            playlists_details.append(
                (
                    item['id'], item['snippet']['title'], item['snippet']['description'],
                    item['contentDetails']['itemCount'], item['status']['privacyStatus']
                )
            )

        nextPageToken = playlists.get('nextPageToken')
        if not nextPageToken:
            break

    return playlists_details

In [9]:
def get_playlist_items(playlistId):
    nextPageToken = None
    video_ids = []
    while True:
        playlistItems = youtube.playlistItems().list(
            part = 'contentDetails',
            playlistId = playlistId,
            pageToken = nextPageToken,
            maxResults = 50
        ).execute()
        
        video_ids += [item['contentDetails']['videoId'] for item in playlistItems['items']]
        
        nextPageToken = playlistItems.get('nextPageToken')
        if not nextPageToken:
            break
    return video_ids